In [ ]:
import pandas as pd
#df = pd.read_table("/content/combined_feature_counts_new.txt", sep='\t', comment='#', skiprows=1).set_index("Geneid").drop(columns = ["Chr","Start","End","Strand","Length"])
#df.columns = ["SRR21043406","SRR21043571","SRR21043800","SRR21043957","SRR21043509","SRR21043270","SRR21043508","SRR21043510"]

# SRR21043510SC.bam	SRR21043270SC.bam	SRR21043955SC.bam	SRR21043508SC.bam	SRR21043798SC.bam	SRR21043509SC.bam	SRR21043569SC.bam	SRR21043404SC.bam
#lis = [ a.replace("SC.bam", "") for a in df.columns]

#df.columns = lis

#df.to_csv("counts_file.txt",header=None, index=None, sep=' ')
df = pd.read_table("/content/TESTcombined_feature_counts.txt").drop(columns = ["Start","End","Strand","Length", "Chr"])
#df.to_csv("new_featurecount_counts_file.txt",header=None, index=None, sep=' ')
df.columns = list(a.replace("SC.bam", "") for a in df.columns)
df.shape

(19568, 769)

In [ ]:
# starting with counts data
import pandas as pd
conditions = pd.read_csv("/content/condition.csv")
counts = df
counts.columns = list(a.replace("SC.bam", "") for a in df.columns)
# print(len({a.split("[")[0] for a in conditions.condition.unique()}))
conditions.condition = conditions.condition.str.strip()
Batch = pd.read_csv("/content/batch.csv")
#sorted(conditions.condition.unique())
#sorted(conditions.condition.to_list())

To extract counts for specific condition you can input the chemical followed by the concentration/s(0.0005–100 µM) if needed.

Here are the conditions present in the experiments:

**CHEMICALS GROUPS**:

BPA   -> BPA_  
4,4-BPF_(Bisphenol_F)  -> 4,4-BPF    
BPAP_(Bisphenol_AP)  -> BPAP     
Bisphenol_S    
BADGE      
<font color = green>Bis_(4-chorophenyl)_Sulfone  -> Sulfone  </font>    
BPAF_(Bisphenol_AF)     
BPS-MAE    
BTUM    
D-8    
Pergafast_201       
2,4-BPS    
TGSA     
BPC_(Bisphenol_C)  -> BPC   
2,4-BPF_(BPF_Analog)     
BPS-MPE        

**CONTROL GROUPS**

DMSO_0.1%   -> DMSO       
cells,_no_treatment_[0]  -> no_treatment       
No_cells;_1X_Lysis_buffer_[0]    -> 1X_Lysis   
No_cells;_Brain_+ve_control_[0] -> Brain      
No_cells;_Universal_human_+ve_control_[0]  -> human        
No_cells;_media_only_[0]   -> media    

**POSITIVE CONTROL**

17B-estradiol  -> estradiol

**NEGATIVE CONTROL**    
Dex   -> Dex


### GOOD SAMPLE CONTROLS:

DMSO -> SRR21043271,SRR21043269,SRR21043270,SRR21043403,

cells no treatment -> SRR21043816,SRR21043814,SRR21043817,SRR21043815,


In [ ]:
import random
import pandas as pd

# Input for treatment group and concentration
treatment = str(input("Enter the treatment group you want to analyze: "))
concentration = r"\["+str(input("Do you want to specify concentration? Enter Q or the concentration value: "))+ r"\]"

# Initialize the condition_treatment and condition_control
condition_treatment = ""
condition_control = "Control"

# Handling concentration input
if "Q" in concentration.upper():
    print("No concentration specified.")
    treatment_dict = conditions[conditions['condition'].str.contains(treatment, case=False, na=False)].set_index('x')['condition'].to_dict()

else:
    treatment_dict = conditions[
    conditions['condition'].str.contains(treatment, case=False, na=False) &
    conditions['condition'].str.contains(concentration, case=False, na=False)
].set_index('x')['condition'].to_dict()
print(treatment_dict)

# Input for control group
control = str(input("Enter the control group you want to compare with: "))
control_samples = conditions[conditions['condition'].str.contains(control, case=False, na=False)]['x'].to_list()
if(control == "DMSO" and len(treatment_dict) == 4):
  control_samples = ["SRR21043816","SRR21043814","SRR21043817","SRR21043815"]
if(("treat" in str(control)) and len(treatment_dict) == 4):
  control_samples = ["SRR21043271","SRR21043269","SRR21043270","SRR21043403"]
copy = treatment_dict.copy()
'''
for i in copy:
  if(str(Batch[Batch.Run == i]["batch"].to_list()[0]) == "Plate 1B index G"):
    print("remove")
    del treatment_dict[i]
copy = control_samples.copy()
for i in copy:
  if(str(Batch[Batch.Run == i]["batch"].to_list()[0]) == "Plate 1B index G"):
    print("remove")
    control_samples.remove(i)
'''
if(len(treatment_dict) < len(control_samples)):
  print("AHHH")
  counts_for_treatment = len(treatment_dict)
  control_samples = random.sample(control_samples, counts_for_treatment)

# Combine treatment and control samples in the same consistent order
filter = ["Geneid"] + list(treatment_dict.keys()) + control_samples

# Filter the counts dataframe to keep only the selected samples (excluding "Geneid")
print(filter)
filter_df = counts[filter]

# Save the filtered counts to a CSV file
filter_df.to_csv("counts_file.csv", index=False)

# Combine treatment and control for consistent ordering
filtered_samples =  list(treatment_dict.keys())  + control_samples

# Filter the batch information for the selected samples
batch_filtered = Batch[Batch['Run'].isin(filtered_samples)]

# Ensure consistent ordering in batch file (treatment first, then control)
batch_filtered = batch_filtered.set_index('Run').loc[filtered_samples].reset_index()

# Save the filtered batch information to a CSV file
batch_filtered.to_csv("batch_file.csv", index=False)



condition_list = treatment_dict.copy()
# Append the control condition for each sample in the control group
for i in control_samples:
    condition_list[i] = control
print(condition_list)

condition_df = pd.DataFrame.from_dict(condition_list, orient='index').reset_index()
condition_df.columns = ['x', 'condition']

# Save the condition file as a tab-separated text file
condition_df.to_csv("condition_file.txt", sep='\t', index=False)

Enter the treatment group you want to analyze: BADGE
Do you want to specify concentration? Enter Q or the concentration value: 0.5
{'SRR21043209': 'BADGE_[0.5]', 'SRR21043291': 'BADGE_[0.5]', 'SRR21043347': 'BADGE_[0.5]', 'SRR21043709': 'BADGE_[0.5]'}
Enter the control group you want to compare with: DMSO
['Geneid', 'SRR21043209', 'SRR21043291', 'SRR21043347', 'SRR21043709', 'SRR21043816', 'SRR21043814', 'SRR21043817', 'SRR21043815']
{'SRR21043209': 'BADGE_[0.5]', 'SRR21043291': 'BADGE_[0.5]', 'SRR21043347': 'BADGE_[0.5]', 'SRR21043709': 'BADGE_[0.5]', 'SRR21043816': 'DMSO', 'SRR21043814': 'DMSO', 'SRR21043817': 'DMSO', 'SRR21043815': 'DMSO'}


In [ ]:
print(condition_df.shape)

(8, 2)


In [ ]:
# Step 1: Define the R code as a string
r_code = """
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")
install.packages("readxl")

# Load necessary libraries
library(DESeq2)
library(ggplot2)
library(tidyverse)
library(readxl)
library(dplyr)

# Read Excel files
counts_data <- read.csv("/content/counts_file.csv")
counts_data <- na.omit(counts_data)

counts_data <- counts_data %>%
  column_to_rownames(var = "Geneid")

if (any(counts_data < 0)) {
  counts_data[counts_data < 0] <- 0
}

print(tail(counts_data))


condition <- read.table("/content/condition_file.txt", header = TRUE, fill = TRUE, row.names = 1)
batch <- read.csv("/content/batch_file.csv", header = TRUE, fill = TRUE, row.names = 1)

conditions <- as.vector(condition[, 1])

# Check if conditions and counts_data have matching column names
if (ncol(counts_data) != length(conditions)) {
  stop("Error: Number of conditions does not match the number of columns in counts_data")
}


colData <- data.frame(row.names = colnames(counts_data), condition = conditions, batch = batch)  # <-- Added batch to colData
colData$condition <- as.factor(colData$condition)

# Ensure "control" is a valid reference level
if ("Control" %in% levels(colData$condition)) {
  colData$condition <- relevel(colData$condition, ref = "Control")
} else {
  stop("Error: 'somthing wrong in condition' is not an existing level in the condition column.")
}

condition_batch <- as.vector(batch[, 1]) # convert to vector
colData$batch <- condition_batch
print(colData)

# Optionally, you can check if condition and batch have the expected number of rows
# print(nrow(condition))
# print(nrow(batch))
print(nrow(counts_data))

dds <- DESeqDataSetFromMatrix(countData = counts_data, colData = colData, design = ~ condition)
print(dds)
# filter2 <- rowSums(counts(dds) >= 10) > (ncol(dds) / 2) ############ remove
# dds <- dds[filter2,]
# Perform VST on the combined data
# VST makes variance  approximately constant across the range of mean values.
# Explanation: Reduces the differences in variability (spread) of gene expression levels between genes that are lowly expressed and those that are highly expressed.
vsd <- vst(dds, blind = FALSE)

vsd_mat <- assay(vsd)
#print(vsd_mat)
# Calculate the Spearman correlation matrix
cor_matrix <- cor(vsd_mat, method = "spearman")
print(cor_matrix)
one_minus_spearman <- 1 - cor_matrix
#print(one_minus_spearman)
# Define the cutoff
cutoff <- 0.1

# Find samples to keep
samples_to_keep <- apply(one_minus_spearman, 2, function(x) any(x < cutoff & x > 0))
# print(length(samples_to_keep)) ----> keeping all as all meet the cutoff limit
# Print names of samples to keep
samples_names_to_keep <- colnames(one_minus_spearman)[samples_to_keep]
#print(samples_names_to_keep)

install.packages("pheatmap")
library(pheatmap)

# Install RColorBrewer if needed
if (!requireNamespace("RColorBrewer", quietly = TRUE)) {
  install.packages("RColorBrewer")
}

# Load the library
library(RColorBrewer)


cor_matrix_copy <- cor_matrix
# Get the condition names from colData
conditions <- colData$condition

# Replace the row and column names in the correlation matrix with condition names
rownames(cor_matrix_copy) <- conditions
colnames(cor_matrix_copy) <- conditions

# Plot the heatmap with condition names on the axes
# Use a color palette from RColorBrewer, such as 'RdBu'
# Custom gradient from blue to white to red
pheatmap(cor_matrix_copy,
         clustering_distance_rows = "correlation",
         clustering_distance_cols = "correlation",
         color = colorRampPalette(c("white","yellow", "pink", "purple"))(100))

condition_lookup <- colData
# Debug: Check the range of correlation matrix values
#summary(as.vector(cor_matrix))

# Check the condition lookup
#head(condition_lookup)

# Re-create correlation dataframe
cor_df <- as.data.frame(as.table(cor_matrix))
names(cor_df) <- c("Sample1", "Sample2", "Correlation")

# Add condition labels
cor_df$Group1 <- condition_lookup[cor_df$Sample1, "condition"]
cor_df$Group2 <- condition_lookup[cor_df$Sample2, "condition"]

# Filter where Sample1 != Sample2
cor_df_filtered <- cor_df %>%
  filter(Sample1 != Sample2)

# Ensure that group names are correctly assigned
cor_df_filtered <- cor_df_filtered %>%
  mutate(Group1 = gsub("^BPA.*", "Treatment", Group1),
         Group2 = gsub("^BPA.*", "Treatment", Group2))

# Debug: Check for any correlation values above 1
#summary(cor_df_filtered$Correlation)

# Check group assignments
#unique(cor_df_filtered$Group1)
#unique(cor_df_filtered$Group2)

# Plot with adjusted binwidth
ggplot(cor_df_filtered, aes(x = Correlation, fill = interaction(Group1, Group2))) +
  geom_histogram(binwidth = 0.01, position = "dodge") +
  labs(title = "Distribution of Correlation Values",
       x = "Spearman Correlation",
       y = "Count") +
  scale_fill_manual(name = "Sample Groups",
                    values = c("Control.Control" = "blue",
                               "Control.Treatment" = "purple",
                               "Treatment.Treatment" = "red",
                               "Treatment.Control" = "pink"),
                    labels = c("Control-Control",
                               "Control-Treatment",
                               "Treatment-Treatment",
                               "Treatment-Control")) +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5),
        axis.text.x = element_text(angle = 45, hjust = 1))

dds <- DESeq(dds)
res <- results(dds)
print(res)
summary(res)

res_sorted_by_padj <- res[order(res$padj), ]
print(res_sorted_by_padj)

# Extract the results from DESeq2 (assuming dds is your DESeq object)
write_res <- results(dds)

# Create a dataframe with gene identifiers and the padj as the 'stat'
df <- data.frame(gene = rownames(write_res), stat = res$padj)

# Sort the dataframe by padj (stat) in descending order
df_sorted <- df[order(df$stat), ]  # Use df$stat for ascending order, -df$stat for descending

# Save the sorted dataframe as a CSV file
write.csv(df_sorted, "gene_stat_for_TreatmentSorted_file.csv", row.names = FALSE)

# creating one with log2fol:

# Create a dataframe with gene identifiers and the log2FoldChange as the 'stat'
df <- data.frame(gene = rownames(write_res), stat = res$log2FoldChange)

# Sort the dataframe by padj (stat) in descending order
df_sorted <- df[order(df$stat), ]  # Use df$stat for ascending order, -df$stat for descending

# Save the sorted dataframe as a CSV file
write.csv(df_sorted, "log2foldBPA100sorted_file.csv", row.names = FALSE)

# creating a file with all stats:

# Create a dataframe with gene identifiers and the log2FoldChange as the 'stat'
df <- data.frame(
  gene = rownames(write_res),      # Gene names
  stat = res$padj,                 # Adjusted p-values
  log2FoldChange = res$log2FoldChange,  # Log2 fold change values
  pvalue = res$pvalue,             # Raw p-values
  baseMean = res$baseMean          # Mean of normalized counts
)


# Sort the dataframe by padj (stat) in descending order
df_sorted <- df[order(df$stat), ]  # Use df$stat for ascending order, -df$stat for descending

# Save the sorted dataframe as a CSV file
write.csv(df_sorted, "fileFORbiostatsquidALLSTAT.csv", row.names = FALSE)

# filtering these values so that I am only looking at genes less than p-adj value 0.01
# Assuming you have already sorted the results by padj
significant_genes <- res[which(!is.na(res$padj) & res$padj < 0.01), ]
#print(significant_genes)
significant_genes_sorted <- significant_genes[order(significant_genes$padj), ]
print(significant_genes_sorted)
# Define the file path where you want to save the CSV file
output_file <- "significant_genes_sorted.csv"

# Save the significant_genes_sorted data frame to a CSV file
write.csv(significant_genes_sorted, file = output_file, row.names = TRUE)

# Assuming dds is your DESeq2 object
normalized_counts <- counts(dds, normalized=TRUE)

write_gct <- function(data_matrix, output_file) {
  # Get the number of genes (rows) and samples (columns)
  num_genes <- nrow(data_matrix)
  num_samples <- ncol(data_matrix)

  # Open file connection
  fileConn <- file(output_file, "w")

  # Write the first two lines (headers)
  writeLines("#1.2", fileConn)  # GCT version
  writeLines(paste(num_genes, num_samples, sep="\t"), fileConn)  # Dimensions of the matrix

  # Prepare the data: add "Name" and "Description" columns
  gene_ids <- rownames(data_matrix)
  description <- rep("NA", num_genes)  # You can modify this if you have descriptions for the genes
  gct_data <- cbind(Name=gene_ids, Description=description, data_matrix)

  # Write the table (gene ids, descriptions, and counts)
  write.table(gct_data, file=fileConn, sep="\t", row.names=FALSE, col.names=TRUE, quote=FALSE)

  # Close the file connection
  close(fileConn)
}

# Call the function to write the GCT file
write_gct(normalized_counts, "deseq_normalized_counts.gct")

# Define the PDF file output
pdf("DESeq2_report.pdf")

# Run DESeq2 analysis
dds <- DESeq(dds)

# Obtain the results (log2 fold changes and p-values)
res <- results(dds)

# MA plot (log fold change vs mean expression)
plotMA(res, main = "MA Plot", ylim = c(-5, 5))

# PCA plot to visualize sample clustering by condition
vsd <- varianceStabilizingTransformation(dds, blind = FALSE)  # or use rlog(dds) for smaller datasets
pca_plot <- plotPCA(vsd, intgroup = "condition")  # No 'main' argument here

# Customize PCA plot title
pca_plot <- pca_plot + ggplot2::ggtitle("PCA Plot by Condition")
print(pca_plot)  # Print the ggplot object

# Dispersion plot to visualize gene variability
plotDispEsts(dds, main = "Dispersion Plot")

# Cook's distance for outlier detection (for one sample)
plot(seq_along(assays(dds)[["cooks"]][,1]), assays(dds)[["cooks"]][,1], type = "h",
     main = "Cook's Distance for Sample 1", xlab = "Genes", ylab = "Cook's Distance")

# Heatmap of sample distances by condition
sampleDists <- dist(t(assay(vsd)))
sampleDistMatrix <- as.matrix(sampleDists)
rownames(sampleDistMatrix) <- colData(dds)$condition  # Ensure conditions are printed instead of accession IDs
colnames(sampleDistMatrix) <- colData(dds)$condition  # Assign conditions to the columns as well
heatmap(sampleDistMatrix, symm = TRUE, main = "Sample Distance Heatmap by Condition")

# Close the PDF device
dev.off()

"""
'''
with open("my_script.R", "w") as r_script:
    r_script.write(r_code)

print("R script created successfully.")

import subprocess

process = subprocess.Popen(['Rscript', 'my_script.R'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Print the output from R in the Colab notebook
print("R Script Output:\n", stdout.decode("utf-8"))
'''

R script created successfully.


KeyboardInterrupt: 

OLD CODE FOR FILE EXTRACTION

In [ ]:
'''import random
import pandas as pd

# Input for treatment group and concentration
treatment = str(input("Enter the treatment group you want to analyze: "))
concentration = r"\["+str(input("Do you want to specify concentration? Enter Q or the concentration value: "))+ r"\]"

# Initialize the condition_treatment and condition_control
condition_treatment = ""
condition_control = "Control"

# Handling concentration input
if "Q" in concentration.upper():
    print("No concentration specified.")
    condition_treatment = treatment
    treatment_samples = conditions[conditions['condition'].str.contains(treatment, case=False, na=False)]['x'].to_list()
else:
    condition_treatment = treatment + "-" + concentration
    print(condition_treatment)
    condition_treatment = "Treatment"
    treatment_samples = conditions[conditions['condition'].str.contains(treatment, case=False, na=False) &
                                   conditions['condition'].str.contains(concentration, case=False, na=False)]['x'].to_list()

# Input for control group
control = str(input("Enter the control group you want to compare with: "))
control_samples = conditions[conditions['condition'].str.contains(control, case=False, na=False)]['x'].to_list()
if(control == "DMSO" and len(treatment_samples) == 4):
  control_samples = ["SRR21043816","SRR21043814","SRR21043817","SRR21043815"]
if(("treat" in str(control)) and len(treatment_samples) == 4):
  control_samples = ["SRR21043271","SRR21043269","SRR21043270","SRR21043403"]
# Randomly select control samples to match the number of treatment samples
# Get the number of treatment samples
# remove problematic samples for estra:
#if("estra" in str(treatment)):
#  for a in ["SRR21043542", "SRR21043541","SRR21043801", "SRR21043800", "SRR21043799", "SRR21043798"]:
#    if(a in treatment_samples):
#      treatment_samples.remove(a)

for i in treatment_samples:
  if(str(Batch[Batch.Run == i]["batch"].to_list()[0]) == "Plate 1B index G"):
    print("remove")
    treatment_samples.remove(i)

if(len(treatment_samples) < len(control_samples)):
  print("AHHH")
  counts_for_treatment = len(treatment_samples)
  control_samples = random.sample(control_samples, counts_for_treatment)

# Combine treatment and control samples in the same consistent order
filter = ["Geneid"] + treatment_samples + control_samples

# Filter the counts dataframe to keep only the selected samples (excluding "Geneid")
print(filter)
filter_df = counts[filter]

# Save the filtered counts to a CSV file
filter_df.to_csv("counts_file.csv", index=False)

# Combine treatment and control for consistent ordering
filtered_samples = treatment_samples + control_samples

# Filter the batch information for the selected samples
batch_filtered = Batch[Batch['Run'].isin(filtered_samples)]

# Ensure consistent ordering in batch file (treatment first, then control)
batch_filtered = batch_filtered.set_index('Run').loc[filtered_samples].reset_index()

# Save the filtered batch information to a CSV file
batch_filtered.to_csv("batch_file.csv", index=False)

# Now create the condition.txt file for the filtered samples
# Create a list to hold condition labels
condition_file_list = []

# Append the treatment condition for each sample in the treatment group
for i in range(len(treatment_samples)):
    condition_file_list.append(condition_treatment)

# Append the control condition for each sample in the control group
for i in range(len(control_samples)):
    condition_file_list.append(condition_control)

# Remove "Geneid" from the filter list (if it exists) before writing the condition file
if "Geneid" in filter:
    filter.remove("Geneid")


# Create a DataFrame with the sample names and their corresponding conditions
condition_df = pd.DataFrame({"x": filter, "condition": condition_file_list})

# Save the condition file as a tab-separated text file
condition_df.to_csv("condition_file.txt", sep='\t', index=False)
'''